<a href="https://colab.research.google.com/github/BatsaikhanTuguldur/KousenProjects/blob/main/chessNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import chess.pgn
sides = []
games = []
length = 100
for i in range(length):
    try:
        if chess.pgn.read_game(pgn).mainline_moves():
            games.append(chess.pgn.read_game(pgn).mainline_moves())
            sides.append(chess.pgn.read_game(pgn).headers["White"])
    except:
        pass
len(games)
100
X = []
y = []
counter2 = 0
for game in games:
    board = chess.Board()
    white = sides[counter2]
    if white == 'Carlsen,Magnus':
        remainder = 0
    else:
        remainder = 1
    counter = 0
    for move in game:
        if counter % 2 == remainder:
            X.append(board.copy())
            y.append(move)
        board.push(move)
        counter += 1
    counter2 += 1
chess_dict = {
    'p' : [1,0,0,0,0,0,0,0,0,0,0,0,0],
    'P' : [0,0,0,0,0,0,1,0,0,0,0,0,0],
    'n' : [0,1,0,0,0,0,0,0,0,0,0,0,0],
    'N' : [0,0,0,0,0,0,0,1,0,0,0,0,0],
    'b' : [0,0,1,0,0,0,0,0,0,0,0,0,0],
    'B' : [0,0,0,0,0,0,0,0,1,0,0,0,0],
    'r' : [0,0,0,1,0,0,0,0,0,0,0,0,0],
    'R' : [0,0,0,0,0,0,0,0,0,1,0,0,0],
    'q' : [0,0,0,0,1,0,0,0,0,0,0,0,0],
    'Q' : [0,0,0,0,0,0,0,0,0,0,1,0,0],
    'k' : [0,0,0,0,0,1,0,0,0,0,0,0,0],
    'K' : [0,0,0,0,0,0,0,0,0,0,0,1,0],
    '.' : [0,0,0,0,0,0,0,0,0,0,0,0,1],
}

def make_matrix(board):
    pgn = board.epd()
    foo = []
    pieces = pgn.split(" ", 1)[0]
    rows = pieces.split("/")
    for row in rows:
        foo2 = []
        for thing in row:
            if thing.isdigit():
                for i in range(0, int(thing)):
                    foo2.append('.')
            else:
                foo2.append(thing)
        foo.append(foo2)
    return foo

def translate(matrix,chess_dict):
    rows = []
    for row in matrix:
        terms = []
        for term in row:
            terms.append(chess_dict[term])
        rows.append(terms)
    return rows
new_chess_dict = {}
for term in chess_dict:
    definition = tuple(chess_dict[term])
    new_chess_dict[definition] = term
    new_chess_dict[term] = definition
import numpy as np
for i in range(len(X)):
    X[i] = translate(make_matrix(X[i]),chess_dict)
def move2array(move):
    from_square = move.from_square
    to_square = move.to_square
    array = np.zeros((64,64))
    array[from_square,to_square] = 1
    return array

def array2move(array):
    from_square = chess.SQUARE_NAMES[int(np.where(array == 1)[0])]
    to_square = chess.SQUARE_NAMES[int(np.where(array == 1)[1])]
    move = chess.Move.from_uci(from_square+to_square)
    return move

array = move2array(y[0]).reshape(64,64)
for i in range(len(y)):
    y[i] = move2array(y[i]).reshape(64,64)
X = np.array(X)
y = np.array(y)
X.shape,y.shape
((3810, 8, 8, 13), (3810, 64, 64))
from keras.models import Sequential
from keras.layers import Dense,BatchNormalization,Dropout,Reshape,Flatten

model = Sequential()
model.add(Dense(32,input_shape = (8,8,13)))
model.add(Dense(64))
model.add(Dense(128))
model.add(Flatten())
model.add(Dense(4096,activation = 'softmax'))
model.add(Reshape((64,64)))

model.compile(optimizer = 'adam',loss = 'mse')
model.fit(X,y)


IndexError: ignored